In [1]:
LABEL_CELL = 'hct116'
DATA_DIR = '/casa/haedong/data/dicer_dsrbd_rescue_hct116_aqseq_youngyoon'

In [77]:
LABEL_CELL = 'hek293t'
DATA_DIR = '/casa/haedong/data/dicer_dsrbd_rescue_hek293t_aqseq_youngyoon'

In [2]:
label_file = pd.read_table('{}/labels.txt'.format(DATA_DIR))

,filepath,label,group,shortname,5_adapter,3_adapter,cell_line
2,raw_files/1-3_1.fq.gz,wt_48_1,WT,WT48_1,GTTCAGAGTTCTACAGTCCGACGATC,TGGAATTCTCGGGTGCCAAGG,HCT116
3,raw_files/1-4_1.fq.gz,wt_48_2,WT,WT48_2,GTTCAGAGTTCTACAGTCCGACGATC,TGGAATTCTCGGGTGCCAAGG,HCT116
6,raw_files/2-3_1.fq.gz,aa_48_1,AA,AA48_1,GTTCAGAGTTCTACAGTCCGACGATC,TGGAATTCTCGGGTGCCAAGG,HCT116
7,raw_files/2-4_1.fq.gz,aa_48_2,AA,AA48_2,GTTCAGAGTTCTACAGTCCGACGATC,TGGAATTCTCGGGTGCCAAGG,HCT116
14,raw_files/4-3_1.fq.gz,del_48_1,DEL,DE48_1,GTTCAGAGTTCTACAGTCCGACGATC,TGGAATTCTCGGGTGCCAAGG,HCT116
15,raw_files/4-4_1.fq.gz,del_48_2,DEL,DE48_2,GTTCAGAGTTCTACAGTCCGACGATC,TGGAATTCTCGGGTGCCAAGG,HCT116


# Abundance

In [3]:
dataframes = []
for label in label_file['label']:
    mirna = pd.read_table('{}/processed_files/{}.bwa_l32_n3.trampoline.merged.filtered.mirna.collapsed.abund'.format(DATA_DIR, label))
    mirna = mirna.set_index('name')
    mirna = mirna.loc[~mirna.index.str.endswith(('offset', 'loop', 'sequence'))]

    spikein = pd.read_table('{}/spike_in/{}.bwa_l32_n3.to_spike_in.unsorted.spike_in.bwa_l32_n1.to_spike_in.unsorted.spike_in.txt'.format(DATA_DIR, label))
    spikein = pd.DataFrame(spikein.groupby('spike_in')['count'].sum())
    
    count = pd.concat([mirna, spikein])
    count.columns = [label]
    dataframes.append(count)
    
count = pd.concat(dataframes, axis=1).fillna(0)
count

/casa/haedong/.local/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,wt_48_1,wt_48_2,aa_48_1,aa_48_2,del_48_1,del_48_2
hsa-let-7a-2-3p,64.000000,49.0,59.0,70.0,37.000000,60.0
hsa-let-7a-3p,1400.500000,1309.5,1243.0,1294.5,593.500000,840.5
hsa-let-7a-5p,53436.666667,47619.0,34535.0,39486.0,14180.666667,19607.0
hsa-let-7b-3p,275.000000,145.0,212.0,235.0,146.000000,210.0
hsa-let-7b-5p,16841.000000,13107.0,14223.0,16364.0,5612.000000,6670.0
...,...,...,...,...,...,...
spike_in_5,686.000000,477.0,679.0,876.0,449.000000,832.0
spike_in_6,338.000000,245.0,377.0,423.0,229.000000,418.0
spike_in_7,525.000000,468.0,448.0,569.0,480.000000,605.0
spike_in_8,527.000000,270.0,419.0,473.0,329.000000,553.0


In [49]:
tmp = count.copy()
tmp.columns = [col.replace('48_', '') for col in tmp.columns]
tmp.to_csv('/casa/haedong/project/dicer_motif/for_publication/rescue.{}.count.txt'.format(LABEL_CELL), sep='\t', header=True, index=True)

In [4]:
is_spike_in = count.index.str.startswith('spike_in')

In [5]:
rpm = count / count.loc[~is_spike_in].sum() * 1000000
rpm

,wt_48_1,wt_48_2,aa_48_1,aa_48_2,del_48_1,del_48_2
hsa-let-7a-2-3p,14.225707,12.078677,18.688470,19.708759,44.848430,44.341684
hsa-let-7a-3p,311.298479,322.796468,393.724876,364.471264,719.393067,621.153089
hsa-let-7a-5p,11877.724417,11738.255079,10939.089790,11117.429385,17188.666034,14490.123290
hsa-let-7b-3p,61.126085,35.743022,67.151789,66.165119,176.969482,155.195894
hsa-let-7b-5p,3743.361429,3230.922727,4505.188188,4607.344741,6802.415997,4929.317200
...,...,...,...,...,...,...
spike_in_5,152.481797,117.582219,215.075777,246.641041,544.241765,614.871351
spike_in_6,75.129515,60.393383,119.416153,119.097215,277.575421,308.913732
spike_in_7,116.695253,115.363686,141.905667,160.204055,581.817477,447.111980
spike_in_8,117.139806,66.555973,132.719810,133.174900,398.787395,408.682521


In [6]:
normalized = rpm / rpm.loc[is_spike_in].sum() * rpm.loc[is_spike_in].sum().min()
normalized

,wt_48_1,wt_48_2,aa_48_1,aa_48_2,del_48_1,del_48_2
hsa-let-7a-2-3p,10.613188,12.078677,10.680463,11.041004,7.780378,7.818090
hsa-let-7a-3p,232.246400,322.796468,225.013831,204.179709,124.801472,109.518409
hsa-let-7a-5p,8861.459109,11738.255079,6251.691608,6228.072618,2981.917554,2554.821474
hsa-let-7b-3p,45.603542,35.743022,38.377258,37.066228,30.700952,27.363315
hsa-let-7b-5p,2792.760892,3230.922727,2574.715788,2581.071274,1180.094117,869.110992
...,...,...,...,...,...,...
spike_in_5,113.760108,117.582219,122.915842,138.170278,94.415941,108.410846
spike_in_6,56.050898,60.393383,68.246351,66.719210,48.154233,54.466026
spike_in_7,87.061307,115.363686,81.099112,89.747590,100.934636,78.832406
spike_in_8,87.392969,66.555973,75.849393,74.605641,69.182282,72.056728


In [7]:
for grp in normalized.columns.str.rsplit('_', 1).str.get(0):
    normalized.loc[:, grp] = normalized.loc[:, normalized.columns.str.startswith(grp)].mean(1)
normalized

,wt_48_1,wt_48_2,aa_48_1,aa_48_2,del_48_1,del_48_2,wt_48,aa_48,del_48
hsa-let-7a-2-3p,10.613188,12.078677,10.680463,11.041004,7.780378,7.818090,11.345932,10.860734,7.799234
hsa-let-7a-3p,232.246400,322.796468,225.013831,204.179709,124.801472,109.518409,277.521434,214.596770,117.159940
hsa-let-7a-5p,8861.459109,11738.255079,6251.691608,6228.072618,2981.917554,2554.821474,10299.857094,6239.882113,2768.369514
hsa-let-7b-3p,45.603542,35.743022,38.377258,37.066228,30.700952,27.363315,40.673282,37.721743,29.032133
hsa-let-7b-5p,2792.760892,3230.922727,2574.715788,2581.071274,1180.094117,869.110992,3011.841809,2577.893531,1024.602555
...,...,...,...,...,...,...,...,...,...
spike_in_5,113.760108,117.582219,122.915842,138.170278,94.415941,108.410846,115.671163,130.543060,101.413394
spike_in_6,56.050898,60.393383,68.246351,66.719210,48.154233,54.466026,58.222141,67.482780,51.310129
spike_in_7,87.061307,115.363686,81.099112,89.747590,100.934636,78.832406,101.212497,85.423351,89.883521
spike_in_8,87.392969,66.555973,75.849393,74.605641,69.182282,72.056728,76.974471,75.227517,70.619505


In [8]:
normalized.loc[:, 'log2_fc_aa_48'] = (normalized['aa_48'] + 1).apply(np.log2) - (normalized['wt_48'] + 1).apply(np.log2)
normalized.loc[:, 'log2_fc_del_48'] = (normalized['del_48'] + 1).apply(np.log2) - (normalized['wt_48'] + 1).apply(np.log2)
normalized

,wt_48_1,wt_48_2,aa_48_1,aa_48_2,del_48_1,del_48_2,wt_48,aa_48,del_48,log2_fc_aa_48,log2_fc_del_48
hsa-let-7a-2-3p,10.613188,12.078677,10.680463,11.041004,7.780378,7.818090,11.345932,10.860734,7.799234,-0.057843,-0.488586
hsa-let-7a-3p,232.246400,322.796468,225.013831,204.179709,124.801472,109.518409,277.521434,214.596770,117.159940,-0.369453,-1.237047
hsa-let-7a-5p,8861.459109,11738.255079,6251.691608,6228.072618,2981.917554,2554.821474,10299.857094,6239.882113,2768.369514,-0.722943,-1.895135
hsa-let-7b-3p,45.603542,35.743022,38.377258,37.066228,30.700952,27.363315,40.673282,37.721743,29.032133,-0.105979,-0.472616
hsa-let-7b-5p,2792.760892,3230.922727,2574.715788,2581.071274,1180.094117,869.110992,3011.841809,2577.893531,1024.602555,-0.224373,-1.554653
...,...,...,...,...,...,...,...,...,...,...,...
spike_in_5,113.760108,117.582219,122.915842,138.170278,94.415941,108.410846,115.671163,130.543060,101.413394,0.173087,-0.188044
spike_in_6,56.050898,60.393383,68.246351,66.719210,48.154233,54.466026,58.222141,67.482780,51.310129,0.209605,-0.179046
spike_in_7,87.061307,115.363686,81.099112,89.747590,100.934636,78.832406,101.212497,85.423351,89.883521,-0.242079,-0.169481
spike_in_8,87.392969,66.555973,75.849393,74.605641,69.182282,72.056728,76.974471,75.227517,70.619505,-0.032690,-0.122649


In [55]:
tmp = normalized.copy()
tmp.columns = [col.replace('_48', '') for col in tmp.columns]
tmp.to_csv('/casa/haedong/project/dicer_motif/for_publication/rescue.{}.spike_in_normalized.txt'.format(LABEL_CELL), sep='\t', header=True, index=True)

# Proportion of the major 5' isomiR

In [9]:
abundance = normalized.loc[~normalized.index.str.startswith('spike_in'), ~normalized.columns.str.contains('log2_fc')]
abundance

,wt_48_1,wt_48_2,aa_48_1,aa_48_2,del_48_1,del_48_2,wt_48,aa_48,del_48
hsa-let-7a-2-3p,10.613188,12.078677,10.680463,11.041004,7.780378,7.818090,11.345932,10.860734,7.799234
hsa-let-7a-3p,232.246400,322.796468,225.013831,204.179709,124.801472,109.518409,277.521434,214.596770,117.159940
hsa-let-7a-5p,8861.459109,11738.255079,6251.691608,6228.072618,2981.917554,2554.821474,10299.857094,6239.882113,2768.369514
hsa-let-7b-3p,45.603542,35.743022,38.377258,37.066228,30.700952,27.363315,40.673282,37.721743,29.032133
hsa-let-7b-5p,2792.760892,3230.922727,2574.715788,2581.071274,1180.094117,869.110992,3011.841809,2577.893531,1024.602555
...,...,...,...,...,...,...,...,...,...
hsa-miR-98-5p,661.334270,952.736426,347.386599,345.425696,154.135600,109.453258,807.035348,346.406148,131.794429
hsa-miR-99a-3p,0.000000,0.000000,0.543074,0.315457,0.210280,0.390904,0.000000,0.429266,0.300592
hsa-miR-99a-5p,12.437330,15.283223,11.223538,11.356461,1.261683,2.866633,13.860276,11.290000,2.064158
hsa-miR-99b-3p,149.082123,174.031544,91.598551,97.791750,3.574768,1.303015,161.556834,94.695150,2.438892


In [10]:
from collections import defaultdict

In [11]:
mature2dataframes = defaultdict(list)
for label in label_file['label']:
    anno = pd.read_table('{}/anno_files/{}.bwa_l32_n3.trampoline.merged.filtered.mirna.gz'.format(DATA_DIR, label))
    anno = anno.loc[~anno['name'].str.endswith(('offset', 'loop', 'sequence'))]
    anno.loc[:, 'mature'] = anno['name'].str.split('|').str.get(2)
    
    start_diff = []
    for index, row in anno.iterrows():
        if row['strand'] == '+': start_diff.append(row['bam_start'] - row['anno_start'])
        elif row['strand'] == '-': start_diff.append(-(row['bam_end'] - row['anno_end']))
        else: raise ValueError()
    anno.loc[:, 'start_diff'] = start_diff
    
    anno = anno[['read_id', 'count', 'mature', 'start_diff', 'read_seq']].drop_duplicates().fillna('')
    anno = anno.loc[anno['start_diff'].isin(range(-5, 6))]
    
    for key, grp in anno.groupby('mature'):
        proportion = pd.DataFrame(grp.groupby('start_diff')['count'].sum() / grp['count'].sum() * 100)
        proportion.columns = [label]
        mature2dataframes[key].append(proportion)

In [12]:
mirnas_over_20 = []
for mature, dataframes in mature2dataframes.items():
    proportion = pd.concat(dataframes, axis=1)
    if (proportion.loc[:, proportion.columns.str.startswith('del')] >= 20).all(1).any(): mirnas_over_20.append(mature)
mirnas_over_20 = list(set(mirnas_over_20))

In [13]:
abundance_wt = abundance.loc[:, abundance.columns.str.startswith('wt')]
abundance_wt = abundance_wt.loc[(abundance_wt != 0).any(1)]
mature2label = abundance_wt.T.idxmax().to_dict()

label2matures = defaultdict(list)
for key, val in mature2label.items():
    label2matures[val].append(key)

In [14]:
mature2first = {}
for label in label_file.loc[label_file['label'].str.startswith('wt')]['label']:
    anno = pd.read_table('{}/anno_files/{}.bwa_l32_n3.trampoline.merged.filtered.mirna.gz'.format(DATA_DIR, label))
    anno = anno.loc[~anno['name'].str.endswith(('offset', 'loop', 'sequence'))]
    anno.loc[:, 'mature'] = anno['name'].str.split('|').str.get(2)
    anno = anno.loc[anno['mature'].isin(label2matures[label])]
    
    start_diff = []
    for index, row in anno.iterrows():
        if row['strand'] == '+': start_diff.append(row['bam_start'] - row['anno_start'])
        elif row['strand'] == '-': start_diff.append(-(row['bam_end'] - row['anno_end']))
        else: raise ValueError()
    anno.loc[:, 'start_diff'] = start_diff
    
    anno = anno[['read_id', 'count', 'mature', 'start_diff', 'read_seq']].drop_duplicates().fillna('')
    
    lines = []
    for key, grp in anno.groupby('mature'):
        first = grp.groupby('start_diff')['count'].sum().idxmax()
        mature2first[key] = first

In [15]:
mature2first

{'hsa-let-7b-3p': 0,
 'hsa-let-7d-3p': 0,
 'hsa-miR-1-3p': 0,
 'hsa-miR-106b-3p': 0,
 'hsa-miR-1180-3p': 0,
 'hsa-miR-1185-5p': 0,
 'hsa-miR-1197': 0,
 'hsa-miR-1202': 1,
 'hsa-miR-1207-3p': -9,
 'hsa-miR-122-5p': 0,
 'hsa-miR-1224-3p': 0,
 'hsa-miR-1224-5p': 0,
 'hsa-miR-1226-3p': 0,
 'hsa-miR-1227-3p': 0,
 'hsa-miR-1228-3p': 0,
 'hsa-miR-1228-5p': 0,
 'hsa-miR-1229-3p': 0,
 'hsa-miR-1229-5p': 0,
 'hsa-miR-1233-3p': -2,
 'hsa-miR-1233-5p': -3,
 'hsa-miR-1234-3p': 0,
 'hsa-miR-1236-5p': 0,
 'hsa-miR-1237-3p': 0,
 'hsa-miR-1244': 11,
 'hsa-miR-1247-5p': 0,
 'hsa-miR-1248': 0,
 'hsa-miR-1249-3p': 0,
 'hsa-miR-1249-5p': 0,
 'hsa-miR-1254': 0,
 'hsa-miR-1255b-2-3p': 1,
 'hsa-miR-1255b-5p': 0,
 'hsa-miR-125b-2-3p': 2,
 'hsa-miR-1260a': 0,
 'hsa-miR-1262': 0,
 'hsa-miR-1268a': 0,
 'hsa-miR-1268b': 0,
 'hsa-miR-127-3p': 0,
 'hsa-miR-1273d': 0,
 'hsa-miR-1273e': 7,
 'hsa-miR-1275': 0,
 'hsa-miR-1276': 0,
 'hsa-miR-128-1-5p': 0,
 'hsa-miR-128-3p': 0,
 'hsa-miR-1284': 0,
 'hsa-miR-1285-5p': 1,
 

In [16]:
dataframes = []
for label in label_file['label']:
    anno = pd.read_table('{}/anno_files/{}.bwa_l32_n3.trampoline.merged.filtered.mirna.gz'.format(DATA_DIR, label))
    anno = anno.loc[~anno['name'].str.endswith(('offset', 'loop', 'sequence'))]
    anno.loc[:, 'mature'] = anno['name'].str.split('|').str.get(2)
    
    start_diff = []
    for index, row in anno.iterrows():
        if row['strand'] == '+': start_diff.append(row['bam_start'] - row['anno_start'])
        elif row['strand'] == '-': start_diff.append(-(row['bam_end'] - row['anno_end']))
        else: raise ValueError()
    anno.loc[:, 'start_diff'] = start_diff
    
    anno = anno[['read_id', 'count', 'mature', 'start_diff', 'read_seq']].drop_duplicates().fillna('')
    anno = anno.loc[anno['start_diff'].isin(range(-5, 6))]
    
    lines = []
    for key, grp in anno.groupby('mature'):
        count_total = grp['count'].sum()
        
        if key in mature2first:
            count_first = grp.loc[grp['start_diff'] == mature2first[key]]['count'].sum()
            proportion = (count_first + 1) / (count_total + 1) * 100
            lines.append([key, proportion])
        
    df = pd.DataFrame(lines)
    df.columns = ['name', label]
    df = df.set_index('name')
    dataframes.append(df)
    
proportion = pd.concat(dataframes, axis=1).dropna()
proportion = proportion.loc[proportion.index.isin(mirnas_over_20)]
proportion

/casa/haedong/.local/lib/python3.6/site-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



,wt_48_1,wt_48_2,aa_48_1,aa_48_2,del_48_1,del_48_2
hsa-let-7a-2-3p,86.538462,77.777778,77.358491,79.365079,78.378378,74.074074
hsa-let-7a-3p,76.656627,76.376696,76.137339,80.759703,65.619224,63.364993
hsa-let-7a-5p,94.683059,95.748235,95.600772,95.383526,97.274442,94.999336
hsa-let-7b-3p,85.915493,86.734694,81.879195,91.666667,88.000000,79.194631
hsa-let-7b-5p,95.958814,96.751758,96.010994,96.084559,97.715510,96.139288
...,...,...,...,...,...,...
hsa-miR-98-3p,79.151943,85.774059,86.868687,89.051095,90.090090,88.648649
hsa-miR-98-5p,96.174176,96.710697,95.597484,95.821855,97.400821,94.825511
hsa-miR-99a-5p,97.368421,93.650794,90.476190,93.150685,71.428571,78.260870
hsa-miR-99b-3p,96.440489,97.595474,95.266272,90.791599,61.111111,90.909091


In [17]:
for sample in proportion.columns[~proportion.columns.str.startswith(('mock', 'wt'))].str.rsplit('_', 1).str.get(0).unique():
    proportion.loc[:, 'log2_fc_{}'.format(sample)] = (proportion.loc[:, proportion.columns.str.startswith(sample)].mean(1) / \
                                                      proportion.loc[:, proportion.columns.str.startswith('wt_48')].mean(1)).apply(np.log2)
proportion

,wt_48_1,wt_48_2,aa_48_1,aa_48_2,del_48_1,del_48_2,log2_fc_aa_48,log2_fc_del_48
hsa-let-7a-2-3p,86.538462,77.777778,77.358491,79.365079,78.378378,74.074074,-0.068253,-0.108116
hsa-let-7a-3p,76.656627,76.376696,76.137339,80.759703,65.619224,63.364993,0.035972,-0.246651
hsa-let-7a-5p,94.683059,95.748235,95.600772,95.383526,97.274442,94.999336,0.004183,0.013891
hsa-let-7b-3p,85.915493,86.734694,81.879195,91.666667,88.000000,79.194631,0.007465,-0.046323
hsa-let-7b-5p,95.958814,96.751758,96.010994,96.084559,97.715510,96.139288,-0.004612,0.008541
...,...,...,...,...,...,...,...,...
hsa-miR-98-3p,79.151943,85.774059,86.868687,89.051095,90.090090,88.648649,0.093099,0.116033
hsa-miR-98-5p,96.174176,96.710697,95.597484,95.821855,97.400821,94.825511,-0.011003,-0.004934
hsa-miR-99a-5p,97.368421,93.650794,90.476190,93.150685,71.428571,78.260870,-0.056941,-0.351745
hsa-miR-99b-3p,96.440489,97.595474,95.266272,90.791599,61.111111,90.909091,-0.060573,-0.352061


In [94]:
tmp = proportion.copy()
tmp.columns = [col.replace('_48', '') for col in tmp.columns]
tmp.to_csv('/casa/haedong/project/dicer_motif/for_publication/rescue.{}.major_5_isomir_proportion.txt'.format(LABEL_CELL), sep='\t', header=True, index=True)